In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re

import folium
from folium import Choropleth

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import KNNImputer

from sklearn.ensemble import RandomForestRegressor

In [22]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('../bin/data_preprocess.csv')

In [24]:
df.sample(10)

,id_provincia,kilometraje,id_concesionario,id_distintivo_ambiental,garantia,precio_contado,precio_nuevo,largo,ancho,alto,capacidad_maletero,num_plazas,batalla,peso,num_puertas,consumo_medio,consumo_carretera,consumo_urbano,deposito,combustible,cilindrada,tipo_cambio,id_traccion,num_marchas,potencia_kw,potencia_cv,par,velocidad_max,aceleracion,mes_matriculacion,ano_matriculacion,co2,num_cilindros,id_sobrealimentacion,id_marca,id_modelo
20171,Madrid,130977,STAR MADRID,ECO,24.0,69990.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,Híbrido Enchufable,NaN,Automático,no disponible,NaN,NaN,320.0,NaN,NaN,NaN,7,2021,NaN,NaN,NaN,MERCEDES-BENZ,CLASE GLE
30614,Murcia,12280,AUDI HUERTAS MOTOR MURCIA,C,12.0,24000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,Gasolina,NaN,Manual,no disponible,NaN,NaN,110.0,NaN,NaN,NaN,8,2023,NaN,NaN,NaN,AUDI,A1
7795,Tarragona,134522,DV SINIESTROS,C,0.0,6599.0,49100.0,4656.0,1890.0,1644.0,550.0,5.0,2873.0,2500.0,5.0,5.0,4.7,5.5,57.0,Diesel,2.143,Automático,total permanente,9.0,125.0,170.0,400.0,210.0,8.3,2,2017,129.0,4.0,Turbo de geometría variable,MERCEDES-BENZ,CLASE GLC
3003,Murcia,5,AUDI HUERTAS MOTOR CARTAGENA,C,48.0,27900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,Gasolina,NaN,Manual,no disponible,NaN,NaN,110.0,NaN,NaN,NaN,12,2023,NaN,NaN,NaN,AUDI,A1
21519,Cantabria,79000,BMW GRÜNBLAU MOTOR,ECO,24.0,NaN,57000.0,4752.0,1918.0,1621.0,525.0,5.0,2864.0,2585.0,5.0,5.0,4.9,5.3,60.0,Diesel,1.995,Automático,total permanente,8.0,140.0,190.0,400.0,205.0,7.9,6,2020,125.0,4.0,Turbo de geometría variable,BMW,X4
10612,Madrid,2877,STAR MADRID,0 EMISIONES,36.0,40990.0,57031.0,4463.0,1834.0,1608.0,340.0,5.0,2729.0,2470.0,5.0,0.0,0.0,0.0,0.0,Eléctrico,NaN,Manual,delantera,1.0,140.0,190.0,385.0,160.0,8.6,5,2024,NaN,NaN,-,MERCEDES-BENZ,EQA
2594,Madrid,54734,Grupo Solera Madrid,ECO,12.0,33500.0,55295.0,4762.0,1847.0,1435.0,495.0,5.0,2820.0,2165.0,5.0,0.0,0.0,0.0,54.0,Diesel,1.968,Automático,delantera,7.0,120.0,163.0,380.0,223.0,8.5,2,2020,NaN,4.0,Turbo,AUDI,A4
32804,Murcia,12280,AUDI HUERTAS MOTOR MURCIA,C,12.0,24000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,Gasolina,NaN,Manual,no disponible,NaN,NaN,110.0,NaN,NaN,NaN,8,2023,NaN,NaN,NaN,AUDI,A1
24309,Madrid,103969,CLICARS MADRID,C,12.0,19990.0,35375.0,4515.0,1865.0,1650.0,591.0,5.0,2680.0,2085.0,5.0,0.0,0.0,0.0,54.0,Gasolina,1.598,Manual,delantera,6.0,112.0,150.0,250.0,189.0,10.3,9,2022,NaN,4.0,Turbo,KIA,SPORTAGE
15760,Madrid,130977,STAR MADRID,ECO,24.0,69990.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,Híbrido Enchufable,NaN,Automático,no disponible,NaN,NaN,320.0,NaN,NaN,NaN,7,2021,NaN,NaN,NaN,MERCEDES-BENZ,CLASE GLE


In [26]:
listado_provincias = {}
for index,detalle in enumerate (df['id_provincia'].unique(), start = 1):
    listado_provincias[index] = detalle

df_provincias = pd.DataFrame.from_dict(listado_provincias, orient='index', columns=['nombre_provincia']).reset_index()
df_provincias.rename(columns={'index': 'id_provincia'}, inplace=True)
df_provincias.to_csv('../bin/listado_provincias.csv', index=False)

conversion_provincias = {v: k for k, v in listado_provincias.items()}
df['id_provincia'] = df['id_provincia'].map(conversion_provincias)

In [28]:
listado_concesionarios = {}
for index,detalle in enumerate (df['id_concesionario'].unique(), start = 1):
    listado_concesionarios[index] = detalle

df_concesionarios = pd.DataFrame.from_dict(listado_concesionarios, orient='index', columns=['nombre_concesionario']).reset_index()
df_concesionarios.rename(columns={'index': 'id_concesionario'}, inplace=True)
df_concesionarios.to_csv('../bin/listado_concesionarios.csv', index=False)

conversion = {v: k for k, v in listado_concesionarios.items()}
df['id_concesionario'] = df['id_concesionario'].map(conversion)

In [30]:
listado_modelos = {}
for index,detalle in enumerate (df['id_modelo'].unique(), start = 1):
    listado_modelos[index] = detalle

df_modelos = pd.DataFrame.from_dict(listado_modelos, orient='index', columns=['nombre_modelo']).reset_index()
df_modelos.rename(columns={'index': 'id_modelo'}, inplace=True)
df_modelos.to_csv('../bin/listado_modelos.csv', index=False)

conversion = {v: k for k, v in listado_modelos.items()}
df['id_modelo'] = df['id_modelo'].map(conversion)

In [32]:
marca_df = pd.read_csv('../bin/listado_marcas.csv')

marca_df['nombre_marca'] = marca_df['nombre_marca'].str.upper()
listado_marcas = marca_df.set_index('id_marca')['nombre_marca'].to_dict()

conversion = {v: k for k, v in listado_marcas.items()}
df['id_marca'] = df['id_marca'].map(conversion)

In [38]:
listado_distintivo_ambiental = {}
for index,detalle in enumerate (df['id_distintivo_ambiental'].unique(), start = 1):
    listado_distintivo_ambiental[index] = detalle
    
df_distintivo_ambiental = pd.DataFrame.from_dict(listado_distintivo_ambiental, orient='index', columns=['nombre_distintivo']).reset_index()
df_distintivo_ambiental.rename(columns={'index': 'id_distintivo'}, inplace=True)
df_distintivo_ambiental.to_csv('../bin/listado_distintivo_ambiental.csv', index=False)

conversion = {v: k for k, v in listado_distintivo_ambiental.items()}
df['id_distintivo_ambiental'] = df['id_distintivo_ambiental'].map(conversion)

In [40]:
listado_combustibles = {}
for index,detalle in enumerate (df['combustible'].unique(), start = 1):
    listado_combustibles[index] = detalle
    
df_combustibles = pd.DataFrame.from_dict(listado_combustibles, orient='index', columns=['nombre_combustible']).reset_index()
df_combustibles.rename(columns={'index': 'id_combustible'}, inplace=True)
df_combustibles.to_csv('../bin/listado_combustibles.csv', index=False)

conversion = {v: k for k, v in listado_combustibles.items()}
df['id_combustible'] = df['combustible'].map(conversion)

df = df.drop(columns=['combustible'])

In [42]:
listado_tipos_cambio = {}
for index,detalle in enumerate (df['tipo_cambio'].unique(), start = 1):
    listado_tipos_cambio[index] = detalle
    
df_tipos_cambio = pd.DataFrame.from_dict(listado_tipos_cambio, orient='index', columns=['nombre_tipo_cambio']).reset_index()
df_tipos_cambio.rename(columns={'index': 'id_tipo_cambio'}, inplace=True)
df_tipos_cambio.to_csv('../bin/listado_tipos_cambio.csv', index=False)

tipos_cambio_to_indices = {v: k for k, v in listado_tipos_cambio.items()}
df['id_tipo_cambio'] = df['tipo_cambio'].map(tipos_cambio_to_indices)

df = df.drop(columns=['tipo_cambio'])

In [44]:
listado_sobrealimentacion = {}
for index,detalle in enumerate (df['id_sobrealimentacion'].unique(), start = 1):
    listado_sobrealimentacion[index] = detalle
    
df_sobrealimentacion = pd.DataFrame.from_dict(listado_sobrealimentacion, orient='index', columns=['nombre_sobrealimentacion']).reset_index()
df_sobrealimentacion.rename(columns={'index': 'id_sobrealimentacion'}, inplace=True)
df_sobrealimentacion.to_csv('../bin/listado_sobrealimentacion.csv', index=False)

conversion = {v: k for k, v in listado_sobrealimentacion.items()}
df['id_sobrealimentacion'] = df['id_sobrealimentacion'].map(conversion)

In [46]:
listado_traccion = {}
for index,detalle in enumerate (df['id_traccion'].unique(), start = 1):
    listado_traccion[index] = detalle
    
df_traccion = pd.DataFrame.from_dict(listado_traccion, orient='index', columns=['nombre_traccion']).reset_index()
df_traccion.rename(columns={'index': 'id_traccion'}, inplace=True)
df_traccion.to_csv('../bin/listado_traccion.csv', index=False)

conversion = {v: k for k, v in listado_traccion.items()}
df['id_traccion'] = df['id_traccion'].map(conversion)

In [49]:
df.head()

,id_provincia,kilometraje,id_concesionario,id_distintivo_ambiental,garantia,precio_contado,precio_nuevo,largo,ancho,alto,capacidad_maletero,num_plazas,batalla,peso,num_puertas,consumo_medio,consumo_carretera,consumo_urbano,deposito,cilindrada,id_traccion,num_marchas,potencia_kw,potencia_cv,par,velocidad_max,aceleracion,mes_matriculacion,ano_matriculacion,co2,num_cilindros,id_sobrealimentacion,id_marca,id_modelo,id_combustible,id_tipo_cambio
0,1,58000,1,1,12.0,19990.0,37250.0,4633.0,1811.0,1429.0,480.0,5.0,2810.0,2045.0,4.0,4.5,4.0,5.3,57.0,1.995,1,6.0,110.0,150.0,320.0,215.0,8.7,11,2018,118.0,4.0,1,8.0,1,1,1
1,1,34680,1,2,12.0,12990.0,19300.0,3571.0,1627.0,1488.0,185.0,4.0,2300.0,1055.0,3.0,0.0,0.0,0.0,35.0,NaN,2,6.0,52.0,70.0,92.0,167.0,13.8,12,2021,NaN,3.0,2,19.0,2,2,1
2,1,66933,2,1,12.0,17390.0,26650.0,4377.0,1806.0,1590.0,430.0,5.0,2646.0,1910.0,5.0,3.8,3.6,4.2,55.0,1.461,2,6.0,81.0,110.0,260.0,182.0,11.9,4,2017,103.0,4.0,2,50.0,3,1,1
3,2,159000,3,1,12.0,8000.0,22425.0,4702.0,1809.0,1499.0,540.0,5.0,2662.0,1985.0,5.0,3.6,3.3,4.2,48.0,1.598,2,6.0,81.0,110.0,300.0,195.0,11.4,1,2017,99.0,4.0,2,52.0,4,1,1
4,1,62695,4,3,12.0,22990.0,44700.0,4545.0,1805.0,1685.0,404.0,5.0,2670.0,2425.0,5.0,0.0,0.0,0.0,43.0,2.360,3,1.0,138.0,188.0,193.0,162.0,10.9,7,2022,NaN,4.0,2,48.0,5,3,1


In [51]:
df.to_csv('../bin/prestaciones.csv', index=False)